In [51]:
import os
import kagglehub

In [52]:
path = kagglehub.dataset_download("omkargurav/face-mask-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/face-mask-dataset


In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [54]:
with_mask_path = '/kaggle/input/face-mask-dataset/data/with_mask'
without_mask_path = '/kaggle/input/face-mask-dataset/data/without_mask'

In [55]:
with_mask_images = [os.path.join(with_mask_path, i) for i in os.listdir(with_mask_path)]
without_mask_images = [os.path.join(without_mask_path, i) for i in os.listdir(without_mask_path)]

In [56]:
with_mask_labels = [1] * len(with_mask_images)
without_mask_labels = [0] * len(without_mask_images)

In [57]:
images = with_mask_images + without_mask_images
labels = with_mask_labels + without_mask_labels
df = pd.DataFrame({'files': images, 'class': labels})
df['class'] = df['class'].astype(str)

In [58]:
df_train, df_temp = train_test_split(df, test_size=0.3, stratify=df['class'], random_state=42)
df_val, df_test = train_test_split(df_temp, test_size=0.5, stratify=df_temp['class'], random_state=42)

In [59]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.25,
    brightness_range=[0.7,1.3],
    horizontal_flip=True,
    fill_mode='nearest'
)

test_val_datagen = ImageDataGenerator(rescale=1./255)

In [60]:
train_gen = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    x_col='files',
    y_col='class',
    target_size=(128, 128),
    class_mode='binary',
    batch_size=32,
    shuffle=True
)

Found 5287 validated image filenames belonging to 2 classes.


In [61]:
val_gen = test_val_datagen.flow_from_dataframe(
    dataframe=df_val,
    x_col='files',
    y_col='class',
    target_size=(128, 128),
    class_mode='binary',
    batch_size=32,
    shuffle=False
)

Found 1133 validated image filenames belonging to 2 classes.


In [62]:
test_gen = test_val_datagen.flow_from_dataframe(
    dataframe=df_test,
    x_col='files',
    y_col='class',
    target_size=(128, 128),
    class_mode='binary',
    batch_size=32,
    shuffle=False
)

Found 1133 validated image filenames belonging to 2 classes.


In [63]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense , Dropout, BatchNormalization , Activation
from tensorflow.keras.regularizers import l2

In [64]:
model = Sequential()

In [65]:
model.add(Conv2D(32, (3, 3), kernel_regularizer=l2(0.001) , padding='same' , input_shape=(128, 128, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.3))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [66]:
model.add(Conv2D(32, (3, 3), kernel_regularizer=l2(0.001) , padding='same' , input_shape=(128, 128, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.3))

In [67]:
model.add(Conv2D(64, (3, 3), kernel_regularizer=l2(0.005) , padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.5))

In [68]:
model.add(Conv2D(128, (3, 3), kernel_regularizer=l2(0.005) , padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.5))

In [69]:
model.add(Flatten())
model.add(Dense(256, kernel_regularizer=l2(0.005)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [70]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [71]:
from keras.callbacks  import EarlyStopping

In [72]:
earlystop = EarlyStopping(patience=2 , restore_best_weights=True)

In [73]:
model.fit(train_gen, validation_data=val_gen, epochs=30 ,  callbacks=[earlystop])

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
 67/166 ━━━━━━━━━━━━━━━━━━━━ 19s 198ms/step - accuracy: 0.6847 - loss: 3.7539

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


166/166 ━━━━━━━━━━━━━━━━━━━━ 60s 274ms/step - accuracy: 0.7311 - loss: 3.3589 - val_accuracy: 0.5066 - val_loss: 2.6711
Epoch 2/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 37s 222ms/step - accuracy: 0.8271 - loss: 1.5722 - val_accuracy: 0.5066 - val_loss: 1.8810
Epoch 3/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 36s 216ms/step - accuracy: 0.8382 - loss: 1.0273 - val_accuracy: 0.5808 - val_loss: 1.2418
Epoch 4/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 36s 216ms/step - accuracy: 0.8408 - loss: 0.8390 - val_accuracy: 0.6002 - val_loss: 1.1257
Epoch 5/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 36s 214ms/step - accuracy: 0.8586 - loss: 0.7010 - val_accuracy: 0.6169 - val_loss: 1.1200
Epoch 6/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 41s 217ms/step - accuracy: 0.8730 - loss: 0.6655 - val_accuracy: 0.7617 - val_loss: 0.7887
Epoch 7/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 36s 220ms/step - accuracy: 0.8858 - loss: 0.6152 - val_accuracy: 0.8561 - val_loss: 0.6495
Epoch 8/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 36s 215ms/step - accuracy: 0.8749 - loss: 0.6212 - val

In [74]:
test_loss, test_acc = model.evaluate(test_gen)
print(f"Test Accuracy: {test_acc*100:.2f}%")

36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - accuracy: 0.9292 - loss: 0.5137
Test Accuracy: 93.03%


In [75]:
from sklearn.metrics import classification_report

y_pred = model.predict(test_gen)
y_pred_classes = (y_pred >= 0.5).astype(int)

y_true = test_gen.classes

print(classification_report(y_true, y_pred_classes, target_names=["Without Mask", "With Mask"]))

36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step
              precision    recall  f1-score   support

Without Mask       0.96      0.90      0.93       574
   With Mask       0.91      0.96      0.93       559

    accuracy                           0.93      1133
   macro avg       0.93      0.93      0.93      1133
weighted avg       0.93      0.93      0.93      1133



In [76]:
model.save('face_mask_detection.h5')